In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# FEATURE EXTRACTION

**we first load the training data from the CSV file using the read_csv function from the pandas library.**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer

# Load the training data
train_df = pd.read_csv('/kaggle/input/deeptweets/train.csv')

**We print it**

In [ ]:
train_df

In [ ]:
tweets = train_df['TweetText'].values
labels = train_df['Label'].values

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit_transform(tweets)
train_bow = vectorizer.transform(tweets)


In [ ]:
print(train_bow.shape)

# **MODEL TRAINING**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lr = LogisticRegression()
lr.fit(train_bow, labels)

In [ ]:
train_preds = lr.predict(train_bow)
train_acc = accuracy_score(labels, train_preds)
print("Training accuracy: {:.2f}".format(train_acc))

# HYPER-PARAMETER TUNING

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.01, 0.1, 1, 10, 100]
}

# Perform grid search with cross-validation to find the best hyperparameters
grid_search = GridSearchCV(lr, param_grid, cv=5, scoring='accuracy')
grid_search.fit(train_bow, labels)

In [ ]:
print("Best hyperparameters: {}".format(grid_search.best_params_))
print("Cross-validation score: {:.2f}".format(grid_search.best_score_))

# Train a logistic regression model with the best hyperparameters on the full training set
best_lr = grid_search.best_estimator_
best_lr.fit(train_bow, labels)

# Evaluate the model on the training data
train_preds = best_lr.predict(train_bow)
train_acc = accuracy_score(labels, train_preds)
print("Training accuracy: {:.2f}".format(train_acc))

# CONCLUSION

In [ ]:
test_df = pd.read_csv('/kaggle/input/deeptweets/test.csv')
test_texts = test_df['TweetText'].tolist()


In [ ]:
test_df

In [ ]:
test_bow = vectorizer.transform(test_texts)

In [ ]:
test_preds = best_lr.predict(test_bow)

In [ ]:
submission_df = pd.DataFrame({'TweetId': test_df['TweetId'], 'Label': test_preds})

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv('submission.csv', index=False)